# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


`torch_dtype` is deprecated! Use `dtype` instead!


/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


`torch_dtype` is deprecated! Use `dtype` instead!
[2025-10-01 10:03:18] `torch_dtype` is deprecated! Use `dtype` instead!


[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0


...

Capturing batches (bs=104 avail_mem=57.16 GB):  15%|█▌        | 3/20 [00:00<00:01,  9.58it/s]

Capturing batches (bs=80 avail_mem=57.14 GB):  25%|██▌       | 5/20 [00:00<00:02,  7.06it/s]

Capturing batches (bs=40 avail_mem=57.12 GB):  50%|█████     | 10/20 [00:01<00:00, 12.78it/s]

Capturing batches (bs=12 avail_mem=57.10 GB):  75%|███████▌  | 15/20 [00:01<00:00, 15.68it/s]

Capturing batches (bs=1 avail_mem=57.08 GB): 100%|██████████| 20/20 [00:01<00:00, 14.51it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Sonia and I am 17 years old. I have a small business and I am a full time student. I have never owned a business before but I want to learn more about it. I have taken a few classes about business but I still have a lot of questions. How can I get started with learning more about business? Can someone provide some resources for me? Here are some resources you could try to get started with learning more about business: 

1. Consider taking some online classes or taking a business course online. You can find many courses online that cover business concepts, accounting, marketing, and more. Some popular options include
Prompt: The president of the United States is
Generated text:  a member of a specific political party. Which political party is it?
A) Democratic
B) Republican
C) Unaffiliated
D) Independent
D) Independent
The president of the United States is a member of the Republican party. While the Democratic and Independent parties are also p

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name], and I'm a [job title] at [company name]. I'm excited to meet you and learn more about your career. What can you tell me about yourself? I'm a [insert a few key points about yourself, such as your personality, skills, or interests]. And what's your background? I'm [insert a few key points about your background, such as your education, work experience, or any relevant skills]. And what's your favorite hobby or activity? I enjoy [insert a few key points about your favorite hobby or activity, such as reading, playing music, or spending time with friends]. And

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, the city known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. It is also home to the French Parliament, the French Academy of Sciences, and the French Quarter, a historic district known for its French colonial architecture and vibrant culture. Paris is a bustling metropolis with a rich history and a diverse population, making it a popular tourist destination. The city is also home to many famous French artists, writers, and musicians, including Pablo Picasso, André Breton, and Claude Monet. Paris is a city of contrasts, with its modern architecture and historical landmarks blending seamlessly

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by several key trends:

1. Increased automation: AI will continue to automate many tasks, from manufacturing and transportation to customer service and healthcare. This will lead to increased efficiency and productivity, but it will also create new job opportunities.

2. Improved privacy and security: As AI systems become more sophisticated, they will need to be designed with privacy and security in mind. This will require ongoing efforts to protect user data and prevent cyber attacks.

3. Enhanced human-computer interaction: AI will continue to improve its ability to understand and respond to human language and emotions. This will lead to more natural and intuitive interactions between humans and



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Name], and I'm a [Role] at [Company]. I've always been [Description of what I do] and I've been [Number of years in this job]. I'm confident in my abilities and I'm always looking for ways to [Describe a problem or challenge you'll be solving or reaching a goal].
Hi there! I'm [Name], a [Description of what I do] at [Company]. I'm [Number of years in this job], and I've always been [Description of what I do]. I'm confident in my abilities and I'm always looking for ways to [Describe a problem or challenge

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris.

That is the capital city of France, which is known for its beautiful architecture, iconic landmarks such as the Eiffel Tower, and a rich cultural heritage. Paris is home to many world-renowned museums, such as the Louvre, and is a popular destination f

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Name

]

 and

...

 could

 lead

 to

 AI

 systems

In [6]:
llm.shutdown()